In [32]:
import json
import string

First combine the lists of raw names from each website scraped with BeautifulSoup:

In [33]:
names = []
for rx_site in ['assist', 'list']:
    with open(f'raw/names_rx{rx_site}.json', 'r') as f:
        names += list(json.load(f))
len(names)

9332

9332 total names. Looking at the contents of the json files there are clearly some duplicates especially when the names involve two words. Generally anything beyond the first word isn't necessarily relevant to the original brand name itself. 

The following saves only the first word for names and removes duplicates. It also makes names all lowercase:

In [34]:
def clean_name(name_str):
    name_str = name_str.split(' ')[0]
    name_str = name_str.lower()
    return name_str

names_clean = list(map(clean_name, names))  # apply the element-wise function
names_clean = list(set(names_clean))        # converting to a set and back to a list will remove duplicates
len(names_clean)

3704

After this step only 3704 names remain. Further filtering may be unwise at this point since we want a sufficiently large dataset. 

The following checks which characters appear in the names other than from the standard English alphabet, to see if there are any special characters:

In [35]:
names_str_long = ''.join(names_clean)
chars = list(
    set(names_str_long).difference(
        set(string.ascii_lowercase)
    )
)
chars

[',', '1', '6', '2', '3', '4', '\t', '5', '8', '7', '.', '0', 'é', '/', '-']

Let's look at these suspect names:

In [36]:
for char in chars:
    print(f'Words with "{char}":')
    print([name for name in names_clean if char in name])
    print('--------')

Words with ",":
['naprosyn,', 'glucophage,', 'prempro,', 'aerobid,', 'biaxin,']
--------
Words with "1":
['hominex-1', 'b12', 'propimex-1', 'vagistat-1', 'i-valex-1', 'glutarex-1', 'niferex-150', 'glofil-125', 'phenex-1', 'cnj-016', 'cyclinex-1', 'ketonex-1', 'tyrex-1']
--------
Words with "6":
['cnj-016', 'md-76r']
--------
Words with "2":
['b12', 'podocon-25', 'i-valex-2', 'ketonex-2', 'tyrex-2', 'cyclinex-2', 'hominex-2', 'glutarex-2', 'acam2000', 'theo-24', 'nordette-28', 'glofil-125', 'mdp-25', 'phenex-2', 'cardiogen-82', 'propimex-2', 'trivora-28']
--------
Words with "3":
['gelsyn-3', 'melquin-3']
--------
Words with "4":
['neotrace-4', 'theo-24', 'kenalog-40']
--------
Words with "	":
['voraxaze\t']
--------
Words with "5":
['anadrol-50', 'podocon-25', 'niferex-150', 'glofil-125', 'mdp-25', 'rimso-50']
--------
Words with "8":
['8-mop', 'nordette-28', 'cardiogen-82', 'trivora-28']
--------
Words with "7":
['md-76r']
--------
Words with ".":
['d.', 'h.p.', 'e.e.s.']
--------
Wor

It seems more duplicates need be removed since the hyphen often acts as an alternative for a space when adding descriptors to the name. To try to best account for the inconcistency of format of the hyphenated names, since sometimes the descriptor tags come before (e.g. `tri-sprintec`) or after (e.g. `neotrace-4`) the presumed drug name, we will assume that the actual drug name in a sequence of hyphenated strings is the largest substring in the sequence.

So, in the previous examples, we would extract the names `sprintec` and `neotrace`:

In [37]:
def clean_hyphens(name_str):
    substrings = name_str.split('-')
    longest = max(substrings, key=len)
    return longest

names_clean = list(map(clean_hyphens, names_clean))     # apply the element-wise function

Let's see what outlier words remain with similar code from earlier:

In [38]:
names_str_long = ''.join(names_clean)
chars = list(
    set(names_str_long).difference(
        set(string.ascii_lowercase)
    )
)

for char in chars:
    print(f'Words with "{char}":', [name for name in names_clean if char in name])

Words with ",": ['naprosyn,', 'glucophage,', 'prempro,', 'aerobid,', 'biaxin,']
Words with "1": ['b12']
Words with "6": ['76r']
Words with "2": ['b12', 'acam2000']
Words with "	": ['voraxaze\t']
Words with "7": ['76r']
Words with ".": ['d.', 'h.p.', 'e.e.s.']
Words with "0": ['acam2000']
Words with "é": ['juvéderm']
Words with "/": ['smoothe/fs']


We make our final cleanup edits based on these words as follows:

In [46]:
def clean_misc(name_str):
    name_str = name_str.strip()     # handle the \t tab character
    name_str = name_str.replace(',', '')       # remove commas from names
    name_str = name_str.replace('é', 'e')      # remove accent on the single name
    return name_str

names_clean = list(map(clean_misc, names_clean))     # apply the element-wise function

# Now remove the rest of the remaining outlier words
names_str_long = ''.join(names_clean)
chars = list(
    set(names_str_long).difference(
        set(string.ascii_lowercase)
    )
)

outlier_words = []
for char in chars:
    outlier_words += [name for name in names_clean if char in name]

names_clean = list(set(names_clean).difference(set(outlier_words)))

Finally we should probably remove names that are excessively short, and make sure we remove any duplicates generated by the cleaning process:

In [49]:
names_clean = [name for name in names_clean if len(name) > 3]
names_final = list(set(names_clean))
len(names_final)

3601

We have 3601 total names to work with. Let's export them to a final JSON file:

In [50]:
with open('names_clean.json', 'w') as f:
    json.dump(names_final, f)